In [1]:
# 1. kütüphaneler
!pip install biopython plotly

import Bio.PDB as PDB
import plotly.graph_objects as go
import numpy as np
import ssl
import os


ssl._create_default_https_context = ssl._create_unverified_context


# Kyte-Doolittle Skalası
KD_SCALE = {
    'ALA': 1.8, 'ARG': -4.5, 'ASN': -3.5, 'ASP': -3.5, 'CYS': 2.5,
    'GLN': -3.5, 'GLU': -3.5, 'GLY': -0.4, 'HIS': -3.2, 'ILE': 4.5,
    'LEU': 3.8, 'LYS': -3.9, 'MET': 1.9, 'PHE': 2.8, 'PRO': -1.6,
    'SER': -0.8, 'THR': -0.7, 'TRP': -0.9, 'TYR': -1.3, 'VAL': 4.2
}

def get_structure(pdb_id):
    print(f"{pdb_id} verisi çekiliyor...")
    pdbl = PDB.PDBList()
    # Colab'de /content klasörüne indirir
    filename = pdbl.retrieve_pdb_file(pdb_id, pdir='/content/pdb', file_format='pdb')
    parser = PDB.PDBParser(QUIET=True)
    return parser.get_structure(pdb_id, filename)

def extract_data(structure):
    x, y, z = [], [], []
    vals, names = [], []


    chain = next(iter(structure[0]))

    for residue in chain:
        if PDB.is_aa(residue):
            try:
                vec = residue['CA'].get_vector()
                x.append(vec[0]); y.append(vec[1]); z.append(vec[2])
                res_name = residue.get_resname()
                names.append(res_name)
                vals.append(KD_SCALE.get(res_name, 0.0))
            except KeyError:
                continue
    return x, y, z, vals, names


pdb_id = "1AQ1"

# Klasör
if not os.path.exists('/content/pdb'):
    os.makedirs('/content/pdb')

try:
    struct = get_structure(pdb_id)
    x, y, z, vals, names = extract_data(struct)

    fig = go.Figure(data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines+markers',
        marker=dict(
            size=5,
            color=vals,
            colorscale='RdBu_r',
            opacity=0.8,
            colorbar=dict(title="Hidrofobiklik")
        ),
        line=dict(color='gray', width=2),
        text=[f"{n}: {v}" for n, v in zip(names, vals)],
        hoverinfo='text'
    )])

    fig.update_layout(
        title=f"Protein {pdb_id} - 3D Hidrofobiklik Analizi",
        scene=dict(bgcolor='black', xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)),
        paper_bgcolor='black',
        font=dict(color='white'),
        margin=dict(l=0, r=0, b=0, t=30),
        height=600
    )

    fig.show()

except Exception as e:
    print(f"Hata: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.0 MB/s eta 0:00:00
1AQ1 verisi çekiliyor...
